In [24]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing
import seaborn as sns 
import optuna 
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score
from sklearn.model_selection import cross_val_score, train_test_split
import os 
from lightgbm import LGBMClassifier

In [2]:
train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/heart_train_data.csv')


In [3]:
test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/heart_test_data.csv')

In [4]:
train_data.head(5)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,HeartDiseaseorAttack
0,1.0,1.0,1.0,27.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,2,2.0,1.0,0.0,0.0,13.0,4,1,0.0
1,1.0,1.0,1.0,25.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,0.0,10.0,5,6,0.0
2,0.0,0.0,1.0,24.0,1.0,0.0,0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,1.0,7.0,3,6,0.0
3,1.0,1.0,1.0,24.0,1.0,0.0,0,0.0,1.0,1.0,...,0.0,2,0.0,30.0,1.0,0.0,11.0,3,3,0.0
4,0.0,0.0,1.0,20.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,1,15.0,0.0,0.0,0.0,2.0,5,5,0.0


In [6]:
train_data.columns

Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'Diabetes', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump',
       'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth',
       'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'HeartDiseaseorAttack'],
      dtype='object')

In [5]:
test_data.head(5)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,HeartDiseaseorAttack
0,1.0,1.0,1.0,30.0,1.0,1.0,0,0.0,0.0,1.0,...,0.0,1,0.0,0.0,0.0,1.0,11.0,5,7,0.0
1,1.0,1.0,1.0,27.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,1,4.0,0.0,0.0,0.0,11.0,5,5,0.0
2,1.0,0.0,1.0,31.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,2,3.0,3.0,0.0,0.0,6.0,5,6,0.0
3,0.0,1.0,1.0,26.0,1.0,0.0,0,1.0,0.0,0.0,...,0.0,2,0.0,0.0,0.0,0.0,12.0,5,5,0.0
4,1.0,1.0,1.0,25.0,1.0,0.0,0,0.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,1.0,9.0,4,6,0.0


In [7]:
train_stage_features=['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'Diabetes', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump',
       'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth',
       'DiffWalk', 'Sex', 'Age', 'Education', 'Income']

test_stage_features=['CholCheck','HvyAlcoholConsump','AnyHealthcare','Veggies','Fruits',
                     'NoDocbcCost','PhysActivity','Education','MentHlth','Smoker']

In [8]:
train_X=train_data.loc[:,train_stage_features] 
train_y=train_data['HeartDiseaseorAttack']
new_test_data_X=test_data[test_stage_features]
new_test_data_y=test_data['HeartDiseaseorAttack']

In [9]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [10]:
# train에는 있는데 test data에는 없는 컬럼 찾기

train_data_columns=np.array(train_X.columns)
test_data_columns=np.array(new_test_data_X.columns)
np.setdiff1d(train_data_columns,test_data_columns)

array(['Age', 'BMI', 'Diabetes', 'DiffWalk', 'GenHlth', 'HighBP',
       'HighChol', 'Income', 'PhysHlth', 'Sex', 'Stroke'], dtype=object)

In [11]:
cat_features = ['Diabetes', 'GenHlth', 'Education','Income']
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
for col in cat_features:    
    train_X[col] = le.fit_transform(train_X[col])

In [12]:
onlytrain_X=train_X.loc[:,['Age', 'BMI', 'Diabetes', 'DiffWalk', 'GenHlth', 'HighBP',
       'HighChol', 'Income', 'PhysHlth', 'Sex', 'Stroke']]

In [13]:
from sklearn.preprocessing import StandardScaler
onlytrain_X_scaled=StandardScaler().fit_transform(onlytrain_X)

In [14]:
from sklearn.decomposition import PCA 
pca=PCA(n_components=1)

In [15]:
pca.fit(onlytrain_X_scaled)

PCA(n_components=1)

In [16]:
onlytrain_X_scaled_pca=pca.transform(onlytrain_X_scaled)

In [17]:
pca_columns=['pca_component1']
onlytrain_pca=pd.DataFrame(onlytrain_X_scaled_pca,columns=pca_columns)

In [18]:
## Train에만 있고 Test data에 없는 피처들을 PCA로 차원축소해주기

onlytrain_pca

,pca_component1
0,1.368744
1,-0.550028
2,-2.051615
3,3.122299
4,-1.975547
...,...
202939,0.455178
202940,4.516945
202941,2.512316
202942,-1.764226


In [19]:
train_X=train_data[test_stage_features] 

In [20]:
new_train_X=pd.concat([train_X,onlytrain_pca],axis=1)


### 베이지안 최적화해주기 - LightgbmRegressor

**1. 5-fold 교차검증 이용해서 Train data로 Validation set을 RMSE가 최저였을 때의 하이퍼파라미터 구하기**

**2. Learning rate 0.01~0.1, max_depth 3~9, n_estimators 100~1000이었을 때에서 가장 최적의 하이퍼파라미터 구하기**

**3. Test data에 새로운 PCA를 하나를 생성해준다.**

In [21]:
import optuna
from lightgbm import LGBMRegressor

# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X, onlytrain_pca['pca_component1'],
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=50)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)

[I 2023-04-25 22:17:21,514] A new study created in memory with name: no-name-c10f415f-796d-4c94-bbca-e63e35614277
/tmp/ipykernel_613886/772960702.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 22:17:35,862] Trial 0 finished with value: 2.178603468112681 and parameters: {'n_estimators': 849, 'max_depth': 9, 'learning_rate': 0.016070381552097612}. Best is trial 0 with value: 2.178603468112681.
/tmp/ipykernel_613886/772960702.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 

{'n_estimators': 672, 'max_depth': 3, 'learning_rate': 0.06719095803819344}


## **Test data 새로운 컬럼 생성하기**

In [22]:
best_lgbmreg=LGBMRegressor(learning_rate=0.06719095803819344,max_depth=3,n_estimators=672,random_state=42)
best_lgbmreg.fit(train_X, onlytrain_pca['pca_component1'])
new_test_data_X['pca_component1']=best_lgbmreg.predict(new_test_data_X[test_stage_features])

/tmp/ipykernel_613886/216768007.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_data_X['pca_component1']=best_lgbmreg.predict(new_test_data_X[test_stage_features])


In [23]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,new_train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-25 22:25:05,911] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_613886/14971935.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 22:25:08,516] Trial 0 finished with value: 0.8104248725225369 and parameters: {'learning_rate': 0.012776529228109535, 'max_depth': 3, 'n_estimators': 215}. Best is trial 0 with value: 0.8104248725225369.
/tmp/ipykernel_613886/14971935.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 22:25:16,434] Trial 1 finished with

Best hyperparameters: {'learning_rate': 0.04495710413799723, 'max_depth': 3, 'n_estimators': 252}
Best AUC: 0.8123


In [25]:

model_lgbm=LGBMClassifier(learning_rate= 0.04495710413799723,max_depth=3, n_estimators=252, scale_pos_weight=ratio,random_state=42)
model_lgbm.fit(new_train_X,train_y)

LGBMClassifier(learning_rate=0.04495710413799723, max_depth=3, n_estimators=252,
               random_state=42, scale_pos_weight=9.617557811028565)

In [27]:
newtest_pred=model_lgbm.predict_proba(new_test_data_X)[:,1]

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.50,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [28]:
get_clf_prob(new_test_data_y,newtest_pred)

임계값:  0.5
오차행렬
[[30361 15596]
 [ 1988  2791]]
정확도: 0.6534, 정밀도: 0.1518, 재현율: 0.5840, F1:0.5082, AUC: 0.6670
